In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

SHEET_NAMES = ["Pre & Post", "Weekly", "Daily"]

In [2]:
df = pd.read_csv("data/class_data.csv", encoding = 'latin-1')
df_interested = df[df["pre_studyinterest"] == "Yes"]

In [4]:
# Verify possuble varnames by checking in bio class variables set
def find_construct_variable(df, codebook, flag):
    
    # construct possible varnames for each Name Stem in its Construct group
    # flag in ["pre", "post", 'd', 'w']
    def build_counstruct_variable_dict(codebook, flag):
        appending = [c for c in list(codebook.columns)[1:list(codebook.columns).index("Construct")]]
        if "pre" in appending:
            construct_var_dict = \
                codebook.groupby("Construct").apply(
                    lambda d: {i: [a + "_" + i for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                               for i in d["Name Stem"]}).to_dict()

        else:
            construct_var_dict = \
                codebook.groupby("Construct").apply(
                    lambda d: {i: [i + flag + str(a) for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                               for i in d["Name Stem"]}).to_dict()
        return construct_var_dict

    construct_var_dict = build_counstruct_variable_dict(codebook, flag)
    return {construct: {var_root: [var for var in construct_var_dict[construct][var_root] + [var_root] if var in df.columns] \
                          for var_root in construct_var_dict[construct]} for construct in construct_var_dict}

def print_non_exist_variable(df, to_ret):
    print("Name Stems that are not in df:  ")
    for construct in to_ret:
        for var_root in to_ret[construct]:
            if len(to_ret[construct][var_root]) == 0:
                print([construct, var_root])
    print('\n')

In [42]:
def build_var_null_percentage_df(df, sheet_name, verbal = False, overwrite = True, student_interested = False, 
                                 count_writer = None, describe_writer = None):
    
    def construct_percentage_df(construct_var_dict):
        df_percentage_master = pd.DataFrame([[construct, name_stem, var] for construct in construct_var_dict 
                                             for name_stem in construct_var_dict[construct] for var in construct_var_dict[construct][name_stem]])
        df_percentage_master.columns = ["Construct", "Name Stem", "Var"]
        df_percentage_master["count"] = df[df_percentage_master["Var"]].count().to_list()
        df_percentage_master["flag"] = [i.replace(j, '').replace('_', '') if i.replace(j, '').replace('_', '') else "original" 
                       for i, j in zip(df_percentage_master["Var"], df_percentage_master["Name Stem"])]
        df_percentage_master = pd.DataFrame(df_percentage_master.pivot(index='Name Stem', columns="flag", values=["count"]))
        return df_percentage_master
    
    def write_count_to_excel(df_percentage, df_codebook, sheet_name = sheet_name, writer = count_writer):
        appending = [c for c in list(df_codebook.columns)[1:list(df_codebook.columns).index("Construct")]]
        df_percentage_level = pd.DataFrame(df_percentage["count"]).reset_index()
        df_percentage_level = df_percentage_level.merge(df_codebook[codebook_vars], 
                                                       left_on = "Name Stem", right_on = "Name Stem", how = "left")   
        if sheet_name == "Pre & Post":
            df_percentage_level[df_percentage_level["original"].notna()].drop(["post", "pre"], axis=1).to_excel(writer, sheet_name = "Pre&Post Original")
            df_percentage_level[df_percentage_level["post"].notna()].drop(["original", "pre"], axis=1).to_excel(writer, sheet_name = "Post")
            df_percentage_level[df_percentage_level["pre"].notna()].drop(["original", "post"], axis=1).to_excel(writer, sheet_name = "Pre")
            df_percentage_level = df_percentage_level[(df_percentage_level["pre"].notna()) & (df_percentage_level["post"].notna())].drop(["original"], axis=1)
            df_percentage_level["retention_rate"] = df_percentage_level["post"] / df_percentage_level["pre"]
            df_percentage_level = df_percentage_level[["Name Stem", "pre", "post", "retention_rate", "Construct", "Item", "Response Values", "Label"]]
            df_percentage_level.to_excel(writer, sheet_name = "Pre & Post")
        else:
            df_percentage_level.to_excel(writer, sheet_name = sheet_name)    
    
    def print_percentage_head(df_percentage, df_codebook):
        df_percentage.columns = ['_'.join(col[::-1]).strip() for col in df_percentage.columns.values]
        df_percentage = pd.DataFrame(df_percentage).reset_index().merge(df_codebook[codebook_vars], 
                                                                   left_on = "Name Stem", right_on = "Name Stem", how = "left")
        df_percentage = df_percentage[codebook_vars + [c for c in df_percentage.columns if c not in codebook_vars]]
        display(df_percentage.head())
        
    def write_description_to_excel(df, construct_var_dict, sheet_name = sheet_name, writer = describe_writer, verbal = verbal):
        found_var = [var for c in construct_var_dict for var_root in construct_var_dict[c] for var in construct_var_dict[c][var_root]]
        df_describe = df[found_var].describe()
        if verbal:
            display(df_describe)
        df_describe.to_excel(writer, sheet_name = sheet_name)
    
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_percentage = construct_percentage_df(construct_var_dict)
    codebook_vars = ["Name Stem", "Construct", "Item", "Response Values", "Label"]
    
    if verbal:
        print(sheet_name, '\n\n')
        print_non_exist_variable(df, construct_var_dict)
        print_percentage_head(df_percentage.copy(), df_codebook)
    
    if overwrite:
        write_count_to_excel(df_percentage.copy(), df_codebook)
        write_description_to_excel(df, construct_var_dict)
  
    return df_percentage
        

## Interested Students

In [43]:
fname_count = "data/student_interested_var_count.xlsx"
fname_describe = "data/student_interested_var_describe.xlsx"
print("Total number of students:  ", df_interested.shape[0])
with pd.ExcelWriter(fname) as count_writer, pd.ExcelWriter(fname_describe) as describe_writer:
    for sheet_name in SHEET_NAMES:
        build_var_null_percentage_df(df_interested, sheet_name, verbal = True, student_interested=True, 
                                     count_writer = count_writer, describe_writer = describe_writer)

Total number of students:   99
Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,post_count,pre_count
0,aca1,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...,NaN,70.0,NaN
1,aca2,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...,NaN,70.0,NaN
2,aca3,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...,NaN,70.0,NaN
3,aca4,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...,NaN,69.0,NaN
4,aca5,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...,NaN,69.0,NaN


,pre_agqmap1,post_agqmap1,pre_agqmap2,post_agqmap2,pre_agqmap5,post_agqmap5,pre_agqmap10,post_agqmap10,pre_agqmap6,post_agqmap6,pre_agqmap17,post_agqmap17,pre_agqmap3,post_agqmap3,pre_agqmap7,post_agqmap7,pre_agqmap16,post_agqmap16,pre_agqmap12,post_activ1,post_activ2,post_activ3,post_activ4,post_activ5,post_activ6,post_activ7,pre_att1,post_att1,pre_att2,post_att2,pre_att3,post_att3,pre_cost1,post_cost1,pre_cost2,post_cost2,pre_cost3,post_cost3,pre_cost4,post_cost4,pre_cost5,post_cost5,pre_cost6,post_cost6,pre_cost7,post_cost7,pre_cost8,post_cost8,pre_cost9,post_cost9,courseformat,coursecode,year,pre_studydays,post_studydays,pre_study,post_study,pre_er1,post_er1,pre_er2,post_er2,pre_er3,post_er3,pre_er4,post_er4,pre_er7x,post_er7x,pre_er8x,post_er8x,pre_essayquestion,post_goalhelpd,post_goalhelpw,post_wgrade,post_egrade,post_badgrade,post_wgradef,post_egradef,post_badgradef,pre_int1,post_int1,pre_int2,post_int2,pre_int3,post_int3,pre_int4,post_int4,pre_iemot1,post_iemot1,pre_iemot2,post_iemot2,pre_iemot3,post_iemot3,pre_iemot4,post_iemot4,pre_iemot5,post_iemot5,pre_iemot6,post_iemot6,pre_iemot7,post_iemot7,pre_onlexp1,pre_onlexp2,pre_orsh1,post_orsh1,pre_orsh2,post_orsh2,pre_orsh3,pre_orsh4,post_orsh4,pre_orsh5,post_orsh5,pre_orsh6,post_orsh6,pre_orsh7,post_orsh7,pre_olsrl1,post_olsrl1,pre_olsrl2,post_olsrl2,pre_olsrl3,post_olsrl3,pre_olsrl4,post_olsrl4,pre_olsrl5,post_olsrl5,pre_courserank,post_courserank,pre_oact1rank,post_oact1rank,pre_oact2rank,post_oact2rank,pre_oact3rank,post_oact3rank,pre_oact4rank,post_oact4rank,pre_oact5rank,post_oact5rank,pre_oact6rank,post_oact6rank,pre_oact7rank,post_oact7rank,pre_oact8rank,post_oact8rank,pre_oact9rank,post_oact9rank,pre_oact10rank,post_oact10rank,post_oact11rank,post_oact12rank,post_oact13rank,post_oact14rank,post_oact15rank,pre_oact1hrs,post_oact1hrs,pre_oact2hrs,post_oact2hrs,pre_oact3hrs,post_oact3hrs,pre_oact4hrs,post_oact4hrs,pre_oact5hrs,post_oact5hrs,pre_oact6hrs,post_oact6hrs,pre_oact7hrs,post_oact7hrs,pre_oact8hrs,post_oact8hrs,pre_oact9hrs,post_oact9hrs,pre_oact10hrs,post_oact10hrs,post_oact11hrs,post_oact12hrs,post_oact13hrs,post_oact14hrs,post_oact15hrs,pre_persist,pre_progress,post_progress,pre_duration,post_duration,pre_finished,post_finished,externalreference,pre_latitude,post_latitude,pre_longitude,post_longitude,rosterid,pre_se1,post_se1,pre_se2,post_se2,pre_se3,post_se3,pre_se4,post_se4,pre_se5,post_se5,pre_util1,post_util1,pre_util2,post_util2,pre_util3,post_util3,pre_util4,post_util4,pre_util5,post_util5
count,99.000000,68.000000,99.000000,68.000000,96.000000,67.000000,94.000000,66.000000,95.000000,63.000000,83.000000,63.000000,96.000000,64.000000,95.000000,69.000000,90.000000,63.000000,82.000000,71.000000,42.000000,54.000000,43.000000,69.000000,53.000000,46.000000,97.000000,66.000000,95.000000,64.000000,93.000000,63.000000,97.000000,67.000000,94.000000,66.000000,91.000000,64.000000,98.000000,68.000000,92.000000,65.000000,93.000000,66.000000,95.000000,68.000000,98.000000,69.000000,97.000000,67.000000,99.0,99.0,99.0,98.000000,71.000000,97.000000,71.000000,82.000000,66.000000,98.000000,70.000000,77.000000,63.000000,97.000000,69.000000,98.000000,68.000000,98.000000,69.000000,0.0,62.000000,62.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,95.000000,66.000000,94.000000,67.000000,95.000000,67.000000,97.000000,67.00000,99.000000,66.000000,97.000000,66.000000,98.000000,66.000000,97.000000,68.000000,95.000000,67.000000,96.000000,69.000000,93.000000,64.000000,99.000000,98.000000,96.000000,70.000000,98.000000,70.000000,97.000000,99.000000,68.000000,99.000000,69.000000,97.000000,68.000000,99.000000,63.000000,97.000000,69.000000,96.000000,69.000000,93.000000,65.000000,93.000000,67.000000,94.000000,70.000000,75.000000,53.000000,70.000000,52.000000,68.000000,49.000000,57.000000,39.000000,42.000000,29.000000,25.000000,17.000000,17.000000,13.000000,11.000000,8.000000,7.000000,4.000000,6.000000,3.0,5.000000,2.00000,21.000000,8.000000,3.0,1.0,1.0,93.000000,65.

Weekly 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,w2_count,w3_count,w4_count,w5_count
0,att3,Attainment Value,How important to your identity is it to be kno...,"slider: 1=Not at all important, 7=Very important",important to my identity to know cooking science,NaN,80.0,71.0,67.0,65.0
1,badgradec,Grade Expectations - Course,Think about your grade in this course...what’s...,slider: 0-100%,worst course grade still satisfactory,NaN,82.0,72.0,69.0,67.0
2,badgradef,Grade Expectations - Final,Think about your grade on the final...what’s t...,slider: 0-100%,worst final grade still satisfactory,NaN,NaN,NaN,69.0,67.0
3,badgradem,Grade Expectations - Midterm,Think about your grade on the midterm...what’s...,slider: 0-100%,worst midterm grade still satisfactory,NaN,82.0,NaN,NaN,NaN
4,badgradeq,Grade Expectations - Quiz,Think about your grade on this week's review q...,slider: 0-100%,worst quiz grade still satisfactory,NaN,82.0,72.0,69.0,67.0


,wcoursechal1,woactchal1,woactchal2,wcoursechal2,woactchal11,woactchal12,wcoursechal3,woactchal21,woactchal22,wcoursechal4,woactchal31,woactchal32,att3w2,att3w3,att3w4,att3w5,wcoursechce1,woactchce1,woactchce2,wcoursechce2,woactchce11,woactchce12,wcoursechce3,woactchce21,woactchce22,wcoursechce4,woactchce31,woactchce32,cost2w2,cost2w3,cost2w4,cost2w5,cost6w2,cost6w3,cost6w4,cost6w5,cost9w2,cost9w3,cost9w4,cost9w5,wcoursecmpl1,woactcmpl1,woactcmpl2,wcoursecmpl2,woactcmpl11,woactcmpl12,wcoursecmpl3,woactcmpl21,woactcmpl22,wcoursecmpl4,woactcmpl31,woactcmpl32,wgradecw2,wgradecw3,wgradecw4,wgradecw5,egradecw2,egradecw3,egradecw4,egradecw5,badgradecw2,badgradecw3,badgradecw4,badgradecw5,wgradefw4,wgradefw5,egradefw4,egradefw5,badgradefw4,badgradefw5,wgrademw2,egrademw2,badgrademw2,wgradeqw2,wgradeqw3,wgradeqw4,wgradeqw5,egradeqw2,egradeqw3,egradeqw4,egradeqw5,badgradeqw2,badgradeqw3,badgradeqw4,badgradeqw5,int3w2,int3w3,int3w4,int3w5,iemot1w2,iemot1w3,iemot1w4,iemot1w5,iemot2w2,iemot2w3,iemot2w4,iemot2w5,iemot3w2,iemot3w3,iemot3w4,iemot3w5,iemot4w2,iemot4w3,iemot4w4,iemot4w5,iemot5w2,iemot5w3,iemot5w4,iemot5w5,iemot7w2,iemot7w3,iemot7w4,iemot7w5,woact28,woact29,woact30,woact37,woact38,woact39,woact40,wcourserank1,woactrank1,woactrank2,woactrank3,woactrank4,woactrank5,woactrank6,woactrank7,woactrank8,woactrank9,woactrank10,wcourserank2,woactrank11,woactrank12,woactrank13,woactrank14,woactrank15,woactrank16,woactrank17,woactrank18,woactrank19,woactrank20,wcourserank3,woactrank21,woactrank22,woactrank23,woactrank24,woactrank25,woactrank26,woactrank27,woactrank28,woactrank29,woactrank30,wcourserank4,woactrank31,woactrank32,woactrank33,woactrank34,woactrank35,woactrank36,woactrank37,woactrank38,woactrank39,woactrank40,wcoursehrs1,woacthrs1,woacthrs2,woacthrs3,woacthrs4,woacthrs5,woacthrs6,woacthrs7,woacthrs8,woacthrs9,woacthrs10,wcoursehrs2,woacthrs11,woacthrs12,woacthrs13,woacthrs14,woacthrs15,woacthrs16,woacthrs17,woacthrs18,woacthrs19,woacthrs20,wcoursehrs3,woacthrs21,woacthrs22,woacthrs23,woacthrs24,woacthrs25,woacthrs26,woacthrs27,woacthrs28,woacthrs29,woacthrs30,wcoursehrs4,woacthrs31,woacthrs32,woacthrs33,woacthrs34,woacthrs35,woacthrs36,woacthrs37,woacthrs38,woacthrs39,woacthrs40,wcourseplan1,woactplan1,woactplan2,wcourseplan2,woactplan11,woactplan12,wcourseplan3,woactplan21,woactplan22,wcourseplan4,woactplan31,woactplan32,progressw2,progressw3,progressw4,progressw5,durationw2,durationw3,durationw4,durationw5,finishedw2,finishedw3,finishedw4,finishedw5,externalreference,latitudew2,latitudew3,latitudew4,latitudew5,longitudew2,longitudew3,longitudew4,longitudew5,rosterid,wcoursehrsx3,wcoursereg1,woactreg1,woactreg2,wcoursereg2,woactreg11,woactreg12,wcoursereg3,woactreg21,woactreg22,wcoursereg4,woactreg31,woactreg32,wcoursesat1,woactsat1,woactsat2,wcoursesat2,woactsat11,woactsat12,wcoursesat3,woactsat21,woactsat22,wcoursesat4,woactsat31,woactsat32,giftcardw2,giftcardw3,giftcardw4,giftcardw5,gettextw3,gettextw4,gettextw5,timezonew2,util1w2,util1w3,util1w4,util1w5,util4w2,util4w3,util4w4,util4w5,util5w2,util5w3,util5w4,util5w5
count,38.000000,38.000000,31.000000,41.000000,41.000000,36.000000,33.000000,25.000000,17.000000,34.000000,33.000000,23.000000,80.000000,71.000000,67.000000,65.000000,37.000000,32.000000,23.000000,39.000000,33.000000,23.000000,24.00000,17.000000,11.000000,32.000000,23.000000,17.000000,75.000000,71.000000,67.000000,65.000000,80.000000,71.000000,68.000000,66.000000,81.000000,71.000000,70.000000,67.000000,37.000000,37.00000,35.000000,40.00000,40.000000,37.000000,33.000000,26.000000,18.000000,34.000000,33.000000,24.000000,82.000000,72.000000,69.000000,67.000000,82.000000,72.000000,69.000000,67.000000,82.000000,72.000000,69.000000,67.000000,69.000000,67.000000,69.000000,67.000000,69.000000,67.000000,82.000000,82.000000,82.000000,82.000000,72.000000,69.000000,67.000000,82.000000,72.000000,69.000000,67.000000,82.000000,72.000000,69.000000,67.000000,81.000000,72.000000,68.000000,66.000000,80.000000,71.000000,67.000000,6

Daily 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phonemodel']
['Study Participation', 'phonecarrier']




,Name Stem,Construct,Item,Response Values,Label,d1_count,d2_count,d3_count,d4_count,d5_count,d6_count,original_count
0,dadd,Reflection on Previous Day Activities,Was there additional course-related activity y...,"0=No, 1=Yes",additional course activity yesterday,NaN,62.0,63.0,62.0,61.0,47.0,NaN
1,daddo,Reflection on Previous Day Activities,"If so, what else did you do?",open-ended,additional course activity - text,NaN,5.0,12.0,6.0,7.0,10.0,NaN
2,dcact1,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 1 (day 1),NaN,NaN,NaN,NaN,NaN,NaN,60.0
3,dcact10,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 10 (day 2),NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,dcact11,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 11 (day 3),NaN,NaN,NaN,NaN,NaN,NaN,57.0


,dcactcompx1,dcactcompx2,dcactcompx3,dcactcompx4,dcactcompx5,dcactcompx6,dcactcompx7,dcactcompx8,dcactcompx9,dcactcompx10,dcactcompx11,dcactcompx12,dcactcompx13,dcactcompx14,dcactcompx15,dcactcompx16,dcactcompx17,dcactcompx18,dcactcompx19,dcactcompx20,dcactcompx21,dcactcompx22,dcactcompx23,dcactcompx24,dcactcompx25,dcactcompx26,dcactcompx27,dcactcompx28,dcactcompx29,dcactcompx30,dcactchal1,dcactchal2,dcactchal3,dcactchal4,dcactchal5,dcactchal6,dcactchal7,dcactchal8,dcactchal9,dcactchal10,dcactchal11,dcactchal12,dcactchal13,dcactchal14,dcactchal15,dcactchal16,dcactchal17,dcactchal18,dcactchal19,dcactchal20,dcactchal21,dcactchal22,dcactchal23,dcactchal24,dcactchal25,dcactchal26,dcactchal27,dcactchal28,dcactchal29,dcactchal30,dcactchce1,dcactchce2,dcactchce3,dcactchce4,dcactchce5,dcactchce6,dcactchce7,dcactchce8,dcactchce9,dcactchce10,dcactchce11,dcactchce12,dcactchce13,dcactchce14,dcactchce15,dcactchce16,dcactchce17,dcactchce18,dcactchce19,dcactchce20,dcactchce21,dcactchce22,dcactchce23,dcactchce24,dcactchce25,dcactchce26,dcactchce27,dcactchce28,dcactchce29,dcactchce30,dcactcost1,dcactcost2,dcactcost3,dcactcost4,dcactcost5,dcactcost6,dcactcost7,dcactcost8,dcactcost9,dcactcost10,dcactcost11,dcactcost12,dcactcost13,dcactcost14,dcactcost15,dcactcost16,dcactcost17,dcactcost18,dcactcost19,dcactcost20,dcactcost21,dcactcost22,dcactcost23,dcactcost24,dcactcost25,dcactcost26,dcactcost27,dcactcost28,dcactcost29,dcactcost30,doact20,doact25,doact30,dcactcomp1,dcactcomp2,dcactcomp3,dcactcomp4,dcactcomp5,dcactcomp6,dcactcomp7,dcactcomp8,dcactcomp9,dcactcomp10,dcactcomp11,dcactcomp12,dcactcomp13,dcactcomp14,dcactcomp15,dcactcomp16,dcactcomp17,dcactcomp18,dcactcomp19,dcactcomp20,dcactcomp21,dcactcomp22,dcactcomp23,dcactcomp24,dcactcomp25,dcactcomp26,dcactcomp27,dcactcomp28,dcactcomp29,dcactcomp30,dcactimp1,dcactimp2,dcactimp3,dcactimp4,dcactimp5,dcactimp6,dcactimp7,dcactimp8,dcactimp9,dcactimp10,dcactimp11,dcactimp12,dcactimp13,dcactimp14,dcactimp15,dcactimp16,dcactimp17,dcactimp18,dcactimp19,dcactimp20,dcactimp21,dcactimp22,dcactimp23,dcactimp24,dcactimp25,dcactimp26,dcactimp27,dcactimp28,dcactimp29,dcactimp30,dcactint1,dcactint2,dcactint3,dcactint4,dcactint5,dcactint6,dcactint7,dcactint8,dcactint9,dcactint10,dcactint11,dcactint12,dcactint13,dcactint14,dcactint15,dcactint16,dcactint17,dcactint18,dcactint19,dcactint20,dcactint21,dcactint22,dcactint23,dcactint24,dcactint25,dcactint26,dcactint27,dcactint28,dcactint29,dcactint30,pland1,pland2,pland3,pland4,pland5,pland6,progressd1,progressd2,progressd3,progressd4,progressd5,progressd6,durationd1,durationd2,durationd3,durationd4,durationd5,durationd6,finishedd1,finishedd2,finishedd3,finishedd4,finishedd5,finishedd6,externalreference,latituded1,latituded2,latituded3,latituded4,latituded5,latituded6,longituded1,longituded2,longituded3,longituded4,longituded5,longituded6,rosterid,dcactrank1,dcactrank2,dcactrank3,dcactrank4,dcactrank5,dcactrank6,dcactrank7,dcactrank8,dcactrank9,dcactrank10,dcactrank11,dcactrank12,dcactrank13,dcactrank14,dcactrank15,dcactrank16,dcactrank17,dcactrank18,dcactrank19,dcactrank20,dcactrank21,dcactrank22,dcactrank23,dcactrank24,dcactrank25,dcactrank26,dcactrank27,dcactrank28,dcactrank29,dcactrank30,doactrank1,doactrank2,doactrank3,doactrank4,doactrank5,doactrank6,doactrank7,doactrank8,doactrank9,doactrank10,doactrank11,doactrank12,doactrank13,doactrank14,doactrank15,doactrank16,doactrank17,doactrank18,doactrank19,doactrank20,doactrank21,doactrank22,doactrank23,doactrank24,doactrank25,doactrank26,doactrank27,doactrank28,doactrank29,doactrank30,dcactcostx1,dcactcostx2,dcactcostx3,dcactcostx4,dcactcostx5,dcactcostx6,dcactcostx7,dcactcostx8,dcactcostx9,dcactcostx10,dcactcostx11,dcactcostx12,dcactcostx13,dcactcostx14,dcactcostx15,dcactcostx16,dcactcostx17,dcactcostx18,dcactcostx19,dcactcostx20,dcactcostx21,dcactcostx22,dcactcostx23,dcactcostx24,dcactcostx25,dcactcostx26,dcactcostx27,dcactcostx28,dcactcostx29,dcactcostx30,dcactimpx1,dcactimpx2,dcactimpx3,dcactimpx4,dcactimpx5,dcact

## All Students

In [19]:
fname = "data/student_var_count.xlsx"
fname_describe = "data/student_var_describe.xlsx"
print("Total number of students:  ", df_interested.shape[0])
with pd.ExcelWriter(fname) as count_writer, pd.ExcelWriter(fname_describe) as describe_writer:
    for sheet_name in SHEET_NAMES:
        build_var_null_percentage_df(df, sheet_name, verbal = True, student_interested=False, 
                                     count_writer = count_writer, describe_writer = describe_writer)

Total number of students:   99
Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,post_count,pre_count
0,aca1,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...,NaN,100.0,NaN
1,aca2,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...,NaN,100.0,NaN
2,aca3,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...,NaN,100.0,NaN
3,aca4,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...,NaN,99.0,NaN
4,aca5,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...,NaN,99.0,NaN


,pre_agqmap1,post_agqmap1,pre_agqmap2,post_agqmap2,pre_agqmap5,post_agqmap5,pre_agqmap10,post_agqmap10,pre_agqmap6,post_agqmap6,pre_agqmap17,post_agqmap17,pre_agqmap3,post_agqmap3,pre_agqmap7,post_agqmap7,pre_agqmap16,post_agqmap16,pre_agqmap12,post_activ1,post_activ2,post_activ3,post_activ4,post_activ5,post_activ6,post_activ7,pre_att1,post_att1,pre_att2,post_att2,pre_att3,post_att3,pre_cost1,post_cost1,pre_cost2,post_cost2,pre_cost3,post_cost3,pre_cost4,post_cost4,pre_cost5,post_cost5,pre_cost6,post_cost6,pre_cost7,post_cost7,pre_cost8,post_cost8,pre_cost9,post_cost9,courseformat,coursecode,year,pre_studydays,post_studydays,pre_study,post_study,pre_er1,post_er1,pre_er2,post_er2,pre_er3,post_er3,pre_er4,post_er4,pre_er7x,post_er7x,pre_er8x,post_er8x,pre_essayquestion,post_goalhelpd,post_goalhelpw,post_wgrade,post_egrade,post_badgrade,post_wgradef,post_egradef,post_badgradef,pre_int1,post_int1,pre_int2,post_int2,pre_int3,post_int3,pre_int4,post_int4,pre_iemot1,post_iemot1,pre_iemot2,post_iemot2,pre_iemot3,post_iemot3,pre_iemot4,post_iemot4,pre_iemot5,post_iemot5,pre_iemot6,post_iemot6,pre_iemot7,post_iemot7,pre_onlexp1,pre_onlexp2,pre_orsh1,post_orsh1,pre_orsh2,post_orsh2,pre_orsh3,pre_orsh4,post_orsh4,pre_orsh5,post_orsh5,pre_orsh6,post_orsh6,pre_orsh7,post_orsh7,pre_olsrl1,post_olsrl1,pre_olsrl2,post_olsrl2,pre_olsrl3,post_olsrl3,pre_olsrl4,post_olsrl4,pre_olsrl5,post_olsrl5,pre_courserank,post_courserank,pre_oact1rank,post_oact1rank,pre_oact2rank,post_oact2rank,pre_oact3rank,post_oact3rank,pre_oact4rank,post_oact4rank,pre_oact5rank,post_oact5rank,pre_oact6rank,post_oact6rank,pre_oact7rank,post_oact7rank,pre_oact8rank,post_oact8rank,pre_oact9rank,post_oact9rank,pre_oact10rank,post_oact10rank,post_oact11rank,post_oact12rank,post_oact13rank,post_oact14rank,post_oact15rank,pre_oact1hrs,post_oact1hrs,pre_oact2hrs,post_oact2hrs,pre_oact3hrs,post_oact3hrs,pre_oact4hrs,post_oact4hrs,pre_oact5hrs,post_oact5hrs,pre_oact6hrs,post_oact6hrs,pre_oact7hrs,post_oact7hrs,pre_oact8hrs,post_oact8hrs,pre_oact9hrs,post_oact9hrs,pre_oact10hrs,post_oact10hrs,post_oact11hrs,post_oact12hrs,post_oact13hrs,post_oact14hrs,post_oact15hrs,pre_persist,pre_progress,post_progress,pre_duration,post_duration,pre_finished,post_finished,externalreference,pre_latitude,post_latitude,pre_longitude,post_longitude,rosterid,pre_se1,post_se1,pre_se2,post_se2,pre_se3,post_se3,pre_se4,post_se4,pre_se5,post_se5,pre_util1,post_util1,pre_util2,post_util2,pre_util3,post_util3,pre_util4,post_util4,pre_util5,post_util5
count,137.000000,95.000000,137.000000,95.000000,134.000000,94.000000,132.000000,93.000000,133.000000,89.00000,117.000000,89.000000,134.000000,90.000000,133.000000,98.000000,125.000000,87.000000,115.000000,102.000000,61.000000,77.000000,59.000000,99.000000,78.000000,69.000000,135.000000,94.000000,132.000000,91.000000,130.000000,90.000000,135.000000,96.000000,132.000000,95.000000,128.00000,92.000000,137.000000,97.000000,127.000000,92.000000,130.000000,93.000000,134.00000,96.000000,137.000000,97.000000,136.000000,96.000000,168.0,168.0,168.0,138.000000,102.000000,137.000000,101.000000,112.000000,93.000000,134.000000,98.000000,106.000000,89.000000,134.000000,97.000000,135.000000,96.000000,135.000000,97.000000,0.0,80.000000,79.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,134.000000,95.000000,133.000000,96.000000,134.000000,96.000000,135.000000,96.000000,137.000000,94.000000,134.000000,94.000000,136.000000,94.000000,135.000000,97.000000,133.000000,96.000000,134.000000,98.000000,130.000000,90.000000,138.000000,137.000000,134.000000,99.000000,135.000000,99.000000,132.000000,137.000000,96.000000,137.000000,98.000000,133.000000,97.000000,135.000000,91.000000,135.000000,98.000000,132.000000,97.000000,128.000000,92.000000,128.000000,95.000000,131.000000,96.000000,106.000000,77.000000,100.000000,72.000000,96.000000,68.000000,78.000000,54.000000,56.000000,38.000000,31.000000,21.000000,18.000000,14.000000,12.000000,9.000000,7.000000,4.000000,6.00000

Weekly 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,w2_count,w3_count,w4_count,w5_count
0,att3,Attainment Value,How important to your identity is it to be kno...,"slider: 1=Not at all important, 7=Very important",important to my identity to know cooking science,NaN,103.0,91.0,83.0,80.0
1,badgradec,Grade Expectations - Course,Think about your grade in this course...what’s...,slider: 0-100%,worst course grade still satisfactory,NaN,106.0,92.0,85.0,82.0
2,badgradef,Grade Expectations - Final,Think about your grade on the final...what’s t...,slider: 0-100%,worst final grade still satisfactory,NaN,NaN,NaN,85.0,82.0
3,badgradem,Grade Expectations - Midterm,Think about your grade on the midterm...what’s...,slider: 0-100%,worst midterm grade still satisfactory,NaN,106.0,NaN,NaN,NaN
4,badgradeq,Grade Expectations - Quiz,Think about your grade on this week's review q...,slider: 0-100%,worst quiz grade still satisfactory,NaN,106.0,92.0,85.0,81.0


,wcoursechal1,woactchal1,woactchal2,wcoursechal2,woactchal11,woactchal12,wcoursechal3,woactchal21,woactchal22,wcoursechal4,woactchal31,woactchal32,att3w2,att3w3,att3w4,att3w5,wcoursechce1,woactchce1,woactchce2,wcoursechce2,woactchce11,woactchce12,wcoursechce3,woactchce21,woactchce22,wcoursechce4,woactchce31,woactchce32,cost2w2,cost2w3,cost2w4,cost2w5,cost6w2,cost6w3,cost6w4,cost6w5,cost9w2,cost9w3,cost9w4,cost9w5,wcoursecmpl1,woactcmpl1,woactcmpl2,wcoursecmpl2,woactcmpl11,woactcmpl12,wcoursecmpl3,woactcmpl21,woactcmpl22,wcoursecmpl4,woactcmpl31,woactcmpl32,wgradecw2,wgradecw3,wgradecw4,wgradecw5,egradecw2,egradecw3,egradecw4,egradecw5,badgradecw2,badgradecw3,badgradecw4,badgradecw5,wgradefw4,wgradefw5,egradefw4,egradefw5,badgradefw4,badgradefw5,wgrademw2,egrademw2,badgrademw2,wgradeqw2,wgradeqw3,wgradeqw4,wgradeqw5,egradeqw2,egradeqw3,egradeqw4,egradeqw5,badgradeqw2,badgradeqw3,badgradeqw4,badgradeqw5,int3w2,int3w3,int3w4,int3w5,iemot1w2,iemot1w3,iemot1w4,iemot1w5,iemot2w2,iemot2w3,iemot2w4,iemot2w5,iemot3w2,iemot3w3,iemot3w4,iemot3w5,iemot4w2,iemot4w3,iemot4w4,iemot4w5,iemot5w2,iemot5w3,iemot5w4,iemot5w5,iemot7w2,iemot7w3,iemot7w4,iemot7w5,woact28,woact29,woact30,woact37,woact38,woact39,woact40,wcourserank1,woactrank1,woactrank2,woactrank3,woactrank4,woactrank5,woactrank6,woactrank7,woactrank8,woactrank9,woactrank10,wcourserank2,woactrank11,woactrank12,woactrank13,woactrank14,woactrank15,woactrank16,woactrank17,woactrank18,woactrank19,woactrank20,wcourserank3,woactrank21,woactrank22,woactrank23,woactrank24,woactrank25,woactrank26,woactrank27,woactrank28,woactrank29,woactrank30,wcourserank4,woactrank31,woactrank32,woactrank33,woactrank34,woactrank35,woactrank36,woactrank37,woactrank38,woactrank39,woactrank40,wcoursehrs1,woacthrs1,woacthrs2,woacthrs3,woacthrs4,woacthrs5,woacthrs6,woacthrs7,woacthrs8,woacthrs9,woacthrs10,wcoursehrs2,woacthrs11,woacthrs12,woacthrs13,woacthrs14,woacthrs15,woacthrs16,woacthrs17,woacthrs18,woacthrs19,woacthrs20,wcoursehrs3,woacthrs21,woacthrs22,woacthrs23,woacthrs24,woacthrs25,woacthrs26,woacthrs27,woacthrs28,woacthrs29,woacthrs30,wcoursehrs4,woacthrs31,woacthrs32,woacthrs33,woacthrs34,woacthrs35,woacthrs36,woacthrs37,woacthrs38,woacthrs39,woacthrs40,wcourseplan1,woactplan1,woactplan2,wcourseplan2,woactplan11,woactplan12,wcourseplan3,woactplan21,woactplan22,wcourseplan4,woactplan31,woactplan32,progressw2,progressw3,progressw4,progressw5,durationw2,durationw3,durationw4,durationw5,finishedw2,finishedw3,finishedw4,finishedw5,externalreference,latitudew2,latitudew3,latitudew4,latitudew5,longitudew2,longitudew3,longitudew4,longitudew5,rosterid,wcoursehrsx3,wcoursereg1,woactreg1,woactreg2,wcoursereg2,woactreg11,woactreg12,wcoursereg3,woactreg21,woactreg22,wcoursereg4,woactreg31,woactreg32,wcoursesat1,woactsat1,woactsat2,wcoursesat2,woactsat11,woactsat12,wcoursesat3,woactsat21,woactsat22,wcoursesat4,woactsat31,woactsat32,giftcardw2,giftcardw3,giftcardw4,giftcardw5,gettextw3,gettextw4,gettextw5,timezonew2,util1w2,util1w3,util1w4,util1w5,util4w2,util4w3,util4w4,util4w5,util5w2,util5w3,util5w4,util5w5
count,47.000000,46.000000,38.000000,52.000000,51.000000,45.000000,39.000000,29.000000,21.000000,43.000000,40.00000,29.000000,103.000000,91.000000,83.000000,80.00000,44.000000,39.000000,29.000000,48.000000,41.000000,27.000000,27.000000,20.000000,13.000000,39.000000,29.000000,22.000000,98.000000,91.000000,83.000000,80.000000,102.000000,90.000000,83.000000,81.000000,105.000000,90.000000,86.000000,82.000000,46.000000,45.000000,43.000000,51.000000,50.000000,46.000000,39.000000,30.000000,22.000000,43.000000,40.000000,30.000000,106.000000,92.000000,85.000000,82.000000,106.000000,92.000000,85.000000,82.000000,106.000000,92.000000,85.000000,82.000000,85.000000,82.000000,85.000000,82.000000,85.000000,82.000000,106.000000,106.000000,106.000000,106.000000,92.000000,85.000000,82.000000,106.000000,92.000000,84.000000,82.000000,106.000000,92.000000,85.000000,81.00000,105.000000,92.000000,84.000000,81.000000,104.000000,91.0000

Daily 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phonemodel']
['Study Participation', 'phonecarrier']




,Name Stem,Construct,Item,Response Values,Label,d1_count,d2_count,d3_count,d4_count,d5_count,d6_count,original_count
0,dadd,Reflection on Previous Day Activities,Was there additional course-related activity y...,"0=No, 1=Yes",additional course activity yesterday,NaN,72.0,77.0,77.0,75.0,56.0,NaN
1,daddo,Reflection on Previous Day Activities,"If so, what else did you do?",open-ended,additional course activity - text,NaN,7.0,16.0,6.0,9.0,13.0,NaN
2,dcact1,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 1 (day 1),NaN,NaN,NaN,NaN,NaN,NaN,77.0
3,dcact10,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 10 (day 2),NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,dcact11,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 11 (day 3),NaN,NaN,NaN,NaN,NaN,NaN,65.0


,dcactcompx1,dcactcompx2,dcactcompx3,dcactcompx4,dcactcompx5,dcactcompx6,dcactcompx7,dcactcompx8,dcactcompx9,dcactcompx10,dcactcompx11,dcactcompx12,dcactcompx13,dcactcompx14,dcactcompx15,dcactcompx16,dcactcompx17,dcactcompx18,dcactcompx19,dcactcompx20,dcactcompx21,dcactcompx22,dcactcompx23,dcactcompx24,dcactcompx25,dcactcompx26,dcactcompx27,dcactcompx28,dcactcompx29,dcactcompx30,dcactchal1,dcactchal2,dcactchal3,dcactchal4,dcactchal5,dcactchal6,dcactchal7,dcactchal8,dcactchal9,dcactchal10,dcactchal11,dcactchal12,dcactchal13,dcactchal14,dcactchal15,dcactchal16,dcactchal17,dcactchal18,dcactchal19,dcactchal20,dcactchal21,dcactchal22,dcactchal23,dcactchal24,dcactchal25,dcactchal26,dcactchal27,dcactchal28,dcactchal29,dcactchal30,dcactchce1,dcactchce2,dcactchce3,dcactchce4,dcactchce5,dcactchce6,dcactchce7,dcactchce8,dcactchce9,dcactchce10,dcactchce11,dcactchce12,dcactchce13,dcactchce14,dcactchce15,dcactchce16,dcactchce17,dcactchce18,dcactchce19,dcactchce20,dcactchce21,dcactchce22,dcactchce23,dcactchce24,dcactchce25,dcactchce26,dcactchce27,dcactchce28,dcactchce29,dcactchce30,dcactcost1,dcactcost2,dcactcost3,dcactcost4,dcactcost5,dcactcost6,dcactcost7,dcactcost8,dcactcost9,dcactcost10,dcactcost11,dcactcost12,dcactcost13,dcactcost14,dcactcost15,dcactcost16,dcactcost17,dcactcost18,dcactcost19,dcactcost20,dcactcost21,dcactcost22,dcactcost23,dcactcost24,dcactcost25,dcactcost26,dcactcost27,dcactcost28,dcactcost29,dcactcost30,doact20,doact25,doact30,dcactcomp1,dcactcomp2,dcactcomp3,dcactcomp4,dcactcomp5,dcactcomp6,dcactcomp7,dcactcomp8,dcactcomp9,dcactcomp10,dcactcomp11,dcactcomp12,dcactcomp13,dcactcomp14,dcactcomp15,dcactcomp16,dcactcomp17,dcactcomp18,dcactcomp19,dcactcomp20,dcactcomp21,dcactcomp22,dcactcomp23,dcactcomp24,dcactcomp25,dcactcomp26,dcactcomp27,dcactcomp28,dcactcomp29,dcactcomp30,dcactimp1,dcactimp2,dcactimp3,dcactimp4,dcactimp5,dcactimp6,dcactimp7,dcactimp8,dcactimp9,dcactimp10,dcactimp11,dcactimp12,dcactimp13,dcactimp14,dcactimp15,dcactimp16,dcactimp17,dcactimp18,dcactimp19,dcactimp20,dcactimp21,dcactimp22,dcactimp23,dcactimp24,dcactimp25,dcactimp26,dcactimp27,dcactimp28,dcactimp29,dcactimp30,dcactint1,dcactint2,dcactint3,dcactint4,dcactint5,dcactint6,dcactint7,dcactint8,dcactint9,dcactint10,dcactint11,dcactint12,dcactint13,dcactint14,dcactint15,dcactint16,dcactint17,dcactint18,dcactint19,dcactint20,dcactint21,dcactint22,dcactint23,dcactint24,dcactint25,dcactint26,dcactint27,dcactint28,dcactint29,dcactint30,pland1,pland2,pland3,pland4,pland5,pland6,progressd1,progressd2,progressd3,progressd4,progressd5,progressd6,durationd1,durationd2,durationd3,durationd4,durationd5,durationd6,finishedd1,finishedd2,finishedd3,finishedd4,finishedd5,finishedd6,externalreference,latituded1,latituded2,latituded3,latituded4,latituded5,latituded6,longituded1,longituded2,longituded3,longituded4,longituded5,longituded6,rosterid,dcactrank1,dcactrank2,dcactrank3,dcactrank4,dcactrank5,dcactrank6,dcactrank7,dcactrank8,dcactrank9,dcactrank10,dcactrank11,dcactrank12,dcactrank13,dcactrank14,dcactrank15,dcactrank16,dcactrank17,dcactrank18,dcactrank19,dcactrank20,dcactrank21,dcactrank22,dcactrank23,dcactrank24,dcactrank25,dcactrank26,dcactrank27,dcactrank28,dcactrank29,dcactrank30,doactrank1,doactrank2,doactrank3,doactrank4,doactrank5,doactrank6,doactrank7,doactrank8,doactrank9,doactrank10,doactrank11,doactrank12,doactrank13,doactrank14,doactrank15,doactrank16,doactrank17,doactrank18,doactrank19,doactrank20,doactrank21,doactrank22,doactrank23,doactrank24,doactrank25,doactrank26,doactrank27,doactrank28,doactrank29,doactrank30,dcactcostx1,dcactcostx2,dcactcostx3,dcactcostx4,dcactcostx5,dcactcostx6,dcactcostx7,dcactcostx8,dcactcostx9,dcactcostx10,dcactcostx11,dcactcostx12,dcactcostx13,dcactcostx14,dcactcostx15,dcactcostx16,dcactcostx17,dcactcostx18,dcactcostx19,dcactcostx20,dcactcostx21,dcactcostx22,dcactcostx23,dcactcostx24,dcactcostx25,dcactcostx26,dcactcostx27,dcactcostx28,dcactcostx29,dcactcostx30,dcactimpx1,dcactimpx2,dcactimpx3,dcactimpx4,dcactimpx5,dcact

## Select Var based on count percentage

In [36]:
def build_decision_df(df, sheet_name, threshold = 0.5, writer = None):
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_master = pd.DataFrame([[construct, name_stem, var] for construct in construct_var_dict 
                              for name_stem in construct_var_dict[construct] for var in construct_var_dict[construct][name_stem]])
    df_master.columns = ["Construct", "Name Stem", "Var"]
    df_master["Percentage"] = df[df_master["Var"]].count().to_list()
    df_master["Percentage"] /= df.shape[0]
    df_master["Decision"] = ["Keep" if i >= threshold else "Drop" for i in df_master["Percentage"]]
    
    if writer:
        df_master.to_excel(writer, sheet_name = sheet_name)
        for dec in ["Keep", "Drop"]:
            df_dec = df_master[df_master["Decision"] == dec]
            df_dec.reset_index(drop=True).to_excel(writer, sheet_name = "{} {}".format(sheet_name, dec))
        df_temp = pd.DataFrame(df_master.groupby("Construct").mean().reset_index())
        df_temp["Decision"] = ["Keep" if i >= threshold else "Drop" for i in df_temp["Percentage"]]
        df_temp.to_excel(writer, sheet_name = "{} AGG Decision".format(sheet_name))

In [37]:
fname = "data/student_interested_var_decision.xlsx"
with pd.ExcelWriter(fname) as decision_writer:
    for sheet_name in SHEET_NAMES:
        build_decision_df(df_interested, sheet_name, writer = decision_writer)